# Preprocess Raw Tables

> N.B. para ejecutar el notebook es necesaro modificar el magic_command de iam_role. Cada cuenta tiene uno asociado, navegar a la pestaña IAM/Roles/LabRole dentro de AWs y copiar ARN como se indica en la imagen adjuntada.
>  

In [6]:
%iam_role arn:aws:iam::484183516222:role/LabRole
%region us-east-1
%number_of_workers 2
%idle_timeout 60
%%configure 
{
  "--conf": "spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog",
  "--datalake-formats": "delta"
}

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
It looks like there is a newer version of the kernel available. The latest version is 1.0.6 and you have 1.0.4 installed.
Please run `pip install --upgrade aws-glue-sessions` to upgrade your kernel
Current iam_role is None
iam_role has been set to arn:aws:iam::484183516222:role/LabRole.
Previous region: None
Setting new region to: us-east-1
Region is set to: us-east-1
Previous number of workers: None
Setting new number of workers to: 2
Current idle_timeout is None minutes.
idle_timeout has been set to 60 minutes.
The following configurations have been updated: {'--conf': 'spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org

In [1]:
spark

Trying to create a Glue session for the kernel.
Session Type: etl
Worker Type: G.1X
Number of Workers: 2
Session ID: 50054946-2353-4a28-8637-ccb4be7f329c
Applying the following default arguments:
--glue_kernel_version 1.0.4
--enable-glue-datacatalog true
--conf spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog
--datalake-formats delta
Waiting for session 50054946-2353-4a28-8637-ccb4be7f329c to get into ready status...
Session 50054946-2353-4a28-8637-ccb4be7f329c has been created.


## 1. imports & Constants

In [2]:
import os
import pyspark.sql.types as t
import pyspark.sql.functions as f

In [3]:
BUCKET_NAME = "s3://vrpoptimiserplatform"
RAW = "raw"
ORDERS = "orders"

BRONZE = "bronze"
SILVER = "silver"
GOLD = "gold"

ADDRESS_DATA = "address_data.json"
CLIENTS_DATA = "client_data.json"
PRODUCTS_DATA = "marketing_sample_for_amazon_com-ecommerce.csv"

ADDRESS_TABLE = "address_table"
CLIENTS_TABLE = "clients_table"
CLIENTS_ADDRESS_TABLE = "clients_address_table"
PRODUCTS_TABLE = "products_table"

RAW_ADDRESS_PATH = os.path.join(BUCKET_NAME, RAW, ADDRESS_DATA)
RAW_CIENTS_PATH = os.path.join(BUCKET_NAME, RAW, CLIENTS_DATA)
RAW_PRODUCTS_PATH = os.path.join(BUCKET_NAME, RAW, PRODUCTS_DATA)

BRONZE_ADDRESS_PATH = os.path.join(BUCKET_NAME, ORDERS, BRONZE, ADDRESS_TABLE)
BRONZE_CLIENTS_PATH = os.path.join(BUCKET_NAME, ORDERS, BRONZE, CLIENTS_TABLE)
BRONZE_PRODUCTS_PATH = os.path.join(BUCKET_NAME, ORDERS, BRONZE, PRODUCTS_TABLE)

SILVER_ADDRESS_PATH = os.path.join(BUCKET_NAME, ORDERS, SILVER, ADDRESS_TABLE)
SILVER_CLIENTS_PATH = os.path.join(BUCKET_NAME, ORDERS, SILVER, CLIENTS_TABLE)
SILVER_PRODUCTS_PATH = os.path.join(BUCKET_NAME, ORDERS, SILVER, PRODUCTS_TABLE)

GOLD_CLIENTS_ADDRESS_PATH = os.path.join(BUCKET_NAME, ORDERS, GOLD, CLIENTS_ADDRESS_TABLE)
GOLD_PRODUCTS_PATH = os.path.join(BUCKET_NAME, ORDERS, GOLD, PRODUCTS_TABLE)

In [4]:
def read_csv_to_df(file_path, schema=None):
    """
    Read CSV file into DataFrame.
    
    :param file_path: Path to the JSON file.
    :param schema: Optional schema to enforce while reading.
    :return: DataFrame
    """
    # Validate file_path
    if not isinstance(file_path, str) or not file_path:
        raise ValueError("Invalid file path provided.")

    # Read DataFrame from CSV
    reader = (
        spark
        .read
        .format("csv")
        .option("header", "true")
        .option("multiLine", "true")
        .option("mode", "PERMISSIVE")
        .option("delimiter", ",")
        .option("escape", '"')
    )
        
    if schema:
        reader = reader.schema(schema)
    else:
        reader = reader.option("inferSchema", "true")
    
    try:
        df = reader.load(file_path)
        return df
    except Exception as e:
        raise IOError(f"Error reading CSV file: {str(e)}")

def read_json_to_df(file_path, schema=None):
    """
    Read JSON file into DataFrame.
    
    :param file_path: Path to the JSON file.
    :param schema: Optional schema to enforce while reading.
    :return: DataFrame
    """
    # Validate file_path
    if not isinstance(file_path, str) or not file_path:
        raise ValueError("Invalid file path provided.")

    # Read DataFrame from JSON
    reader = spark.read.format("json").option("multiLine", "true").option("mode", "PERMISSIVE")
    
    if schema:
        reader = reader.schema(schema)
    else:
        reader = reader.option("inferSchema", "true")
    
    try:
        df = reader.load(file_path)
        return df
    except Exception as e:
        raise IOError(f"Error reading JSON file: {str(e)}")

def write_df_to_metastore(df, file_path, table_name, partition_by=None, mode="overwrite"):
    """
    Write DataFrame to Parquet file and save it as a table in the metastore.
    
    :param df: DataFrame to be written.
    :param file_path: Path where the Parquet file will be saved.
    :param table_name: Name of the table to save.
    :param partition_by: Column(s) to partition by.
    :param mode: Write mode, default is 'overwrite'. Other options are 'append', 'ignore', 'error'.
    """
    # Validate parameters
    if not file_path or not isinstance(file_path, str):
        raise ValueError("Invalid file path provided.")
    if not table_name or not isinstance(table_name, str):
        raise ValueError("Invalid table name provided.")
    if partition_by and not isinstance(partition_by, (str, list)):
        raise ValueError("Partition by should be a string or a list of strings.")

    writer = df.write.format("parquet").mode(mode).option("path", file_path)
    
    if partition_by:
        writer = writer.partitionBy(partition_by)
    
    writer.saveAsTable(table_name)

def transform_clients_bronze_to_silver(clients_df):
    """
    Transform Bronze (raw) Clients DataFrame to Silver (cleaned) DataFrame.
    
    :param clients_df: Clients DataFrame.
    :return: Cleaned Clients DataFrame.
    """
    # Example transformations: Filtering active clients, renaming columns, etc.
    clients_silver_df = (
        clients_df
        .filter(f.col("status") == "active")
    )
    return clients_silver_df

def transform_addresses_bronze_to_silver(addresses_df):
    """
    Transform Bronze (raw) Addresses DataFrame to Silver (cleaned) DataFrame.
    
    :param addresses_df: Addresses DataFrame.
    :return: Cleaned Addresses DataFrame.
    """
    # Cast coordenates format to float
    addresses_silver_df = (
        addresses_df
        #.filter(col("house_number") != "")
        .withColumn("lat", f.col("lat").cast("float"))
        .withColumn("lon", f.col("lon").cast("float"))
    )
    return addresses_silver_df


def transform_clients_addresses_silver_to_gold(clients_silver_df, addresses_silver_df):
    """
    Transform Silver (cleaned) Clients and Addresses DataFrames to Gold (aggregated/enriched) DataFrame.
    
    :param clients_silver_df: Cleaned Clients DataFrame.
    :param addresses_silver_df: Cleaned Addresses DataFrame.
    :return: Enriched DataFrame combining both clients and addresses.
    """
    # Example aggregation: Joining clients with their addresses
    gold_df = (
        clients_silver_df
        .join(addresses_silver_df, on="client_id", how="right")
        .dropna(subset=['client_id'])
    )
    return gold_df

In [5]:
def pounds_to_kg(pounds):
    try:
        return float(pounds) * 0.45359237
    except ValueError:
        return None

def ounces_to_kg(ounces):
    try:
        return float(ounces) * 0.0283495
    except ValueError:
        return None



def transform_products_bronze_to_silver(products_df, min_stock=75):
    """
    Transform Bronze (raw) Products DataFrame to Silver (cleaned) DataFrame.
    
    :param products_df: Products DataFrame.
    :return: Cleaned Productss DataFrame.
    """
    # Declare UDF function
    convert_to_kg_udf = f.udf(lambda x: convert_to_kg(x), t.DoubleType())
    # Define UDFs
    pounds_to_kg_udf = f.udf(pounds_to_kg, t.FloatType())
    ounces_to_kg_udf = f.udf(ounces_to_kg, t.FloatType())
    products_silver_df = (
        products_df
        # Filter missing values from 
        .filter(f.col("Shipping Weight").isNotNull())
        # Add prefix to product_id column
        .withColumn("product_id", f.concat(f.lit("pro-"), f.col("Uniq Id")))
        # Cast Price column
        .withColumn("price", f.regexp_replace(f.col("Selling Price"), r'\$', ''))
        # Cast & Convert weight units to kg
        .withColumn("cleaned_weight", f.trim(f.col("Shipping Weight")))
        .withColumn("value", f.regexp_extract(f.col("cleaned_weight"), r"(\d+\.?\d*)", 1))
        .withColumn("unit", f.regexp_extract(f.col("cleaned_weight"), r"(pounds|ounces)", 1))
        .withColumn(
            "weight",
            f.when(f.col("unit") == "pounds", f.round(pounds_to_kg_udf(f.col("value")),2))
            .when(f.col("unit") == "ounces", f.round(ounces_to_kg_udf(f.col("value")),2))
            .otherwise(None)
        )
        # Add Random Quantity column
        .withColumn('quantity', (min_stock + f.round(f.rand(seed=43)*42, 0)))
        # Add timestamp
        .withColumn("updated_at", f.current_timestamp())
        # select 
        .select(
            f.col("product_id"), 
            f.col("Product Name").alias("product_name"), 
            f.col("Category").alias("category"),
            f.col("price").cast(t.DoubleType()),
            f.col("weight").cast(t.DoubleType()),
            f.col("quantity").cast(t.IntegerType()),
            f.col("Product Specification").alias("product_details"),
            f.col("updated_at")
        )
        # Drop rows with NaN values in 'price' or 'weight' columns
        .dropna(subset=["price", "weight"], how="any")
    )
    return products_silver_df

## 2. Medallion Architecture

### 2.1 Bronze Layer

In [6]:
# Define the schema for the clients JSON structure
clients_schema = t.StructType([
    t.StructField("client_id", t.StringType(), False),
    t.StructField("first_name", t.StringType(), False),
    t.StructField("last_name", t.StringType(), False),
    t.StructField("email", t.StringType(), True),
    t.StructField("phone_number", t.StringType(), True),
    t.StructField("date_of_birth", t.DateType(), True),
    t.StructField("gender", t.StringType(), True),
    t.StructField("occupation", t.StringType(), True),
    t.StructField("created_at", t.DateType(), True),
    t.StructField("updated_at", t.DateType(), True),
    t.StructField("status", t.StringType(), True)
])

# Define the schema for the addresses JSON structure
addresses_schema = t.StructType([
    t.StructField("client_id", t.StringType(), False),
    t.StructField("address_id", t.StringType(), False),
    t.StructField("neighborhood", t.StringType(), True),
    t.StructField("coordinates", t.ArrayType(t.DoubleType()), True),
    t.StructField("road", t.StringType(), True),
    t.StructField("house_number", t.StringType(), True),
    t.StructField("suburb", t.StringType(), True),
    t.StructField("city_district", t.StringType(), True),
    t.StructField("state", t.StringType(), True),
    t.StructField("postcode", t.StringType(), True),
    t.StructField("country", t.StringType(), True),
    t.StructField("lat", t.StringType(), True),
    t.StructField("lon", t.StringType(), True)
])

# Define the schema for the products CSV structure
products_schema = t.StructType([
    t.StructField('Uniq Id', t.StringType(), True), 
    t.StructField('Product Name', t.StringType(), True), 
    t.StructField('Brand Name', t.StringType(), True), 
    t.StructField('Asin', t.StringType(), True), 
    t.StructField('Category', t.StringType(), True), 
    t.StructField('Upc Ean Code', t.StringType(), True), 
    t.StructField('List Price', t.StringType(), True), 
    t.StructField('Selling Price', t.StringType(), True), 
    t.StructField('Quantity', t.StringType(), True), 
    t.StructField('Model Number', t.StringType(), True), 
    t.StructField('About Product', t.StringType(), True), 
    t.StructField('Product Specification', t.StringType(), True), 
    t.StructField('Technical Details', t.StringType(), True), 
    t.StructField('Shipping Weight', t.StringType(), True), 
    t.StructField('Product Dimensions', t.StringType(), True), 
    t.StructField('Image', t.StringType(), True), 
    t.StructField('Variants', t.StringType(), True), 
    t.StructField('Sku', t.StringType(), True), 
    t.StructField('Product Url', t.StringType(), True), 
    t.StructField('Stock', t.StringType(), True), 
    t.StructField('Product Details', t.StringType(), True), 
    t.StructField('Dimensions', t.StringType(), True), 
    t.StructField('Color', t.StringType(), True), 
    t.StructField('Ingredients', t.StringType(), True), 
    t.StructField('Direction To Use', t.StringType(), True), 
    t.StructField('Is Amazon Seller', t.StringType(), True), 
    t.StructField('Size Quantity Variant', t.StringType(), True), 
    t.StructField('Product Description', t.StringType(), True)
])

In [7]:
df_address_raw = read_json_to_df(RAW_ADDRESS_PATH, addresses_schema)
df_clients_raw = read_json_to_df(RAW_CIENTS_PATH, clients_schema)
df_products_raw = read_csv_to_df(RAW_PRODUCTS_PATH, products_schema)

In [8]:
write_df_to_metastore(df_address_raw, BRONZE_ADDRESS_PATH, "bronze_address_table")
write_df_to_metastore(df_clients_raw, BRONZE_CLIENTS_PATH, "bronze_clients_table")
write_df_to_metastore(df_products_raw, BRONZE_PRODUCTS_PATH, "bronze_products_table")

### 2.2 Silver Layer

In [9]:
df_address_bronze = spark.table("bronze_address_table")
df_clients_bronze = spark.table("bronze_clients_table")
df_products_bronze = spark.table("bronze_products_table")

In [10]:
write_df_to_metastore(transform_addresses_bronze_to_silver(df_address_bronze), SILVER_ADDRESS_PATH, "silver_address_table")
write_df_to_metastore(transform_clients_bronze_to_silver(df_clients_bronze), SILVER_CLIENTS_PATH, "silver_clients_table")
write_df_to_metastore(transform_products_bronze_to_silver(df_products_bronze), SILVER_PRODUCTS_PATH, "silver_products_table")

### 2.3 Gold Layer

In [11]:
df_address_silver = spark.table("silver_address_table")
df_clients_silver = spark.table("silver_clients_table")
df_products_silver = spark.table("silver_products_table")

In [12]:
write_df_to_metastore(transform_clients_addresses_silver_to_gold(df_clients_silver, df_address_silver), GOLD_CLIENTS_ADDRESS_PATH, "gold_clients_address_table")
write_df_to_metastore(df_products_silver, GOLD_PRODUCTS_PATH, "gold_products_table")

In [13]:
df_products_silver.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- price: double (nullable = true)
 |-- weight: double (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- product_details: string (nullable = true)
 |-- updated_at: timestamp (nullable = true)
